# 从原样本中生成新样本
- 方法：
  利用解决样本不平衡的SMOTE算法
- 步骤：
  1. 从原始五个表中提取特征
  2. 利用最终的特征总表进行SMOTE样本生成
  3. 提取SMOTE样本中的关键信息
  4. 从SMOTE样本关键信息中倒推新样本的浏览记录
  5. 从SMOTE样本关键信息中倒推新样本的银行流水记录
  6. 从原样本的用户基本信息表中随机抽取生成新样本的用户基本信息表

In [ ]:
import numpy as np 
import pandas as pd 
import time
import datetime as dt
from imblearn.over_sampling import SMOTE
import math
import random
from random import sample
from random import randrange
from collections import Counter

#   特征工程
目的：提取特征，利用SMOTE算法来进行样本的生成



## 银行流水记录表

In [ ]:
#---------------------------读取数据 ------------------------------------------
银行流水记录=pd.read_csv("../data/train/bank_detail_train.txt",header=None,
                    names=['用户标识','流水时间','交易类型','交易金额','工资收入标记'])
银行流水记录_test=pd.read_csv("../data/test/bank_detail_test.txt",header=None,
                    names=['用户标识','流水时间','交易类型','交易金额','工资收入标记'])
银行流水记录=pd.concat([银行流水记录,银行流水记录_test],axis=0)

#将时间戳转换为真实日期形式提取月份
银行流水记录['流水日期'] = 银行流水记录['流水时间'].apply(lambda x :time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x)))
银行流水记录['流水日期']=pd.to_datetime(银行流水记录['流水日期'])
银行流水记录['month'] = 银行流水记录['流水日期'].apply(lambda x :x.strftime('%m'))

#划分  收入 支出 工资
银行流水记录_收入=银行流水记录[银行流水记录["交易类型"]==0][["用户标识","流水时间","交易金额",'month']]
银行流水记录_支出=银行流水记录[银行流水记录["交易类型"]==1][["用户标识","流水时间","交易金额",'month']]
银行流水记录_工资=银行流水记录[银行流水记录["工资收入标记"]==1][["用户标识","流水时间","交易金额",'month']]

银行流水记录_收入=银行流水记录_收入.rename(columns={'交易金额':'收入金额'})
银行流水记录_支出=银行流水记录_支出.rename(columns={'交易金额':'支出金额'})
银行流水记录_工资=银行流水记录_工资.rename(columns={'交易金额':'工资金额'})

In [ ]:
#-------------------总体特征提取----------
gb1=银行流水记录_收入.groupby(["用户标识"],as_index=False)#收入统计
gb2=银行流水记录_支出.groupby(["用户标识"],as_index=False)#支出统计
gb3=银行流水记录_工资.groupby(["用户标识"],as_index=False)#工资收入统计
x1=gb1['收入金额'].agg({'用户收入笔数' : 'count','用户收入总计':'sum','用户收入均值':'mean','用户收入标准差':'std'})
x2=gb2['支出金额'].agg({'用户支出笔数' : 'count','用户支出总计':'sum','用户支出均值':'mean','用户支出标准差':'std'})
x3=gb3['工资金额'].agg({'用户工资笔数' : 'count','用户工资总计':'sum','用户工资均值':'mean','用户工资标准差':'std'})

print(len(x1),len(x2),len(x3))
feature=pd.merge(x1, x2,how='outer', on = "用户标识")
feature=pd.merge(feature, x3,how='outer', on = "用户标识")

In [ ]:
#———————————————收入 month维度下的特征提取---------------------
d=银行流水记录_收入

#-----month维度下的特征提取
#sum
收入金额_sum =d['收入金额'].groupby([d['用户标识'],d['month']]).sum()
收入金额_sum=pd.DataFrame(收入金额_sum.unstack())

name_dic={}
for i in 收入金额_sum.columns:
    name_dic[str(i)]='收入金额_sum_month_'+str(i)
收入金额_sum=收入金额_sum.rename(columns=name_dic)

#mean
收入金额_means =d['收入金额'].groupby([d['用户标识'],d['month']]).mean()
收入金额_means=pd.DataFrame(收入金额_means.unstack())

name_dic={}
for i in 收入金额_means.columns:
    name_dic[str(i)]='收入金额_mean_month_'+str(i)
收入金额_means=收入金额_means.rename(columns=name_dic)

#std
收入金额_std =d['收入金额'].groupby([d['用户标识'],d['month']]).std()
收入金额_std=pd.DataFrame(收入金额_std.unstack())

name_dic={}
for i in 收入金额_std.columns:
    name_dic[str(i)]='收入金额_std_month_'+str(i)
收入金额_std=收入金额_std.rename(columns=name_dic)
#max
收入金额_max =d['收入金额'].groupby([d['用户标识'],d['month']]).max()
收入金额_max=pd.DataFrame(收入金额_max.unstack())

name_dic={}
for i in 收入金额_max.columns:
    name_dic[str(i)]='收入金额_max_month_'+str(i)
收入金额_max=收入金额_max.rename(columns=name_dic)
#min
收入金额_min =d['收入金额'].groupby([d['用户标识'],d['month']]).min()
收入金额_min=pd.DataFrame(收入金额_min.unstack())

name_dic={}
for i in 收入金额_min.columns:
    name_dic[str(i)]='收入金额_min_month_'+str(i)
收入金额_min=收入金额_min.rename(columns=name_dic)

银行流水收入_month=pd.merge(收入金额_sum,收入金额_means,how="inner",on="用户标识")
银行流水收入_month=pd.merge(银行流水收入_month,收入金额_max,how="inner",on="用户标识")
银行流水收入_month=pd.merge(银行流水收入_month,收入金额_min,how="inner",on="用户标识")
银行流水收入_month=pd.merge(银行流水收入_month,收入金额_std,how="inner",on="用户标识")

In [ ]:
#------------------------------支出  month维度下的特征提取---------------------------
d=银行流水记录_支出

#sum
支出金额_sum =d['支出金额'].groupby([d['用户标识'],d['month']]).sum()
支出金额_sum=pd.DataFrame(支出金额_sum.unstack())

name_dic={}
for i in 支出金额_sum.columns:
    name_dic[str(i)]='支出金额_sum_month_'+str(i)
支出金额_sum=支出金额_sum.rename(columns=name_dic)

#mean
支出金额_mean =d['支出金额'].groupby([d['用户标识'],d['month']]).mean()
支出金额_means=pd.DataFrame(支出金额_mean.unstack())

name_dic={}
for i in 支出金额_means.columns:
    name_dic[str(i)]='支出金额_mean_month_'+str(i)
支出金额_means=支出金额_means.rename(columns=name_dic)

#std
支出金额_std =d['支出金额'].groupby([d['用户标识'],d['month']]).std()
支出金额_std=pd.DataFrame(支出金额_std.unstack())

name_dic={}
for i in 支出金额_std.columns:
    name_dic[str(i)]='支出金额_std_month_'+str(i)
支出金额_std=支出金额_std.rename(columns=name_dic)
#max
支出金额_max =d['支出金额'].groupby([d['用户标识'],d['month']]).max()
支出金额_max=pd.DataFrame(支出金额_max.unstack())

name_dic={}
for i in 支出金额_max.columns:
    name_dic[str(i)]='支出金额_max_month_'+str(i)
支出金额_max=支出金额_max.rename(columns=name_dic)
#min
支出金额_min =d['支出金额'].groupby([d['用户标识'],d['month']]).min()
支出金额_min=pd.DataFrame(支出金额_min.unstack())

name_dic={}
for i in 支出金额_min.columns:
    name_dic[str(i)]='支出金额_min_month_'+str(i)
支出金额_min=支出金额_min.rename(columns=name_dic)

银行流水支出_month=pd.merge(支出金额_sum,支出金额_means,how="inner",on="用户标识")
银行流水支出_month=pd.merge(银行流水支出_month,支出金额_max,how="inner",on="用户标识")
银行流水支出_month=pd.merge(银行流水支出_month,支出金额_min,how="inner",on="用户标识")
银行流水支出_month=pd.merge(银行流水支出_month,支出金额_std,how="inner",on="用户标识")


In [ ]:
#------------------------------工资 month维度下的特征提取------------------------------------
d=银行流水记录_工资
    
#sum
工资金额_sum =d['工资金额'].groupby([d['用户标识'],d['month']]).sum()
工资金额_sum=pd.DataFrame(工资金额_sum.unstack())

name_dic={}
for i in 工资金额_sum.columns:
    name_dic[str(i)]='工资金额_sum_month_'+str(i)
工资金额_sum=工资金额_sum.rename(columns=name_dic)

#mean
工资金额_mean =d['工资金额'].groupby([d['用户标识'],d['month']]).mean()
工资金额_mean=pd.DataFrame(工资金额_mean.unstack())

name_dic={}
for i in 工资金额_mean.columns:
    name_dic[str(i)]='工资金额_mean_month_'+str(i)
工资金额_means=工资金额_mean.rename(columns=name_dic)

#std
工资金额_std =d['工资金额'].groupby([d['用户标识'],d['month']]).std()
工资金额_std=pd.DataFrame(工资金额_std.unstack())

name_dic={}
for i in 工资金额_std.columns:
    name_dic[str(i)]='工资金额_std_month_'+str(i)
工资金额_std=工资金额_std.rename(columns=name_dic)
#max
工资金额_max =d['工资金额'].groupby([d['用户标识'],d['month']]).max()
工资金额_max=pd.DataFrame(工资金额_max.unstack())

name_dic={}
for i in 工资金额_max.columns:
    name_dic[str(i)]='工资金额_max_month_'+str(i)
工资金额_max=工资金额_max.rename(columns=name_dic)
#min
工资金额_min =d['工资金额'].groupby([d['用户标识'],d['month']]).min()
工资金额_min=pd.DataFrame(工资金额_min.unstack())

name_dic={}
for i in 工资金额_min.columns:
    name_dic[str(i)]='工资金额_min_month_'+str(i)
工资金额_min=工资金额_min.rename(columns=name_dic)

银行流水工资_month=pd.merge(工资金额_sum,工资金额_means,how="inner",on="用户标识")
银行流水工资_month=pd.merge(银行流水工资_month,工资金额_max,how="inner",on="用户标识")
银行流水工资_month=pd.merge(银行流水工资_month,工资金额_min,how="inner",on="用户标识")
银行流水工资_month=pd.merge(银行流水工资_month,工资金额_std,how="inner",on="用户标识")


In [ ]:
bank_feature=pd.merge(feature,银行流水收入_month,how='outer',on='用户标识')
bank_feature=pd.merge(bank_feature,银行流水支出_month,how="outer",on="用户标识")
bank_feature=pd.merge(bank_feature,银行流水工资_month,how="outer",on="用户标识")

##   信用卡账单记录表

In [ ]:
#读取文件
bill= pd.read_csv("bill_detail.csv")

In [ ]:
d=bill
###补充特征，增加整体的各种统计信息sum count max min mean std var等
gb=d.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
                                  '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度','还款状态']].groupby(["用户标识"],as_index=False)

整体账单sum=gb.sum()
整体账单sum.columns = ['用户标识', '整体上期账单金额sum', '整体上期还款金额sum','整体信用卡额度sum','整体本期账单余额sum',
                     '整体本期账单最低还款额sum','整体消费笔数sum','整体本期账单金额sum','整体调整金额sum','整体循环利息sum',
                     '整体可用余额sum','整体预借现金额度sum','整体还款状态sum']
feature=整体账单sum
feature['整体上期还款金额sum与整体上期账单金额sum差值']=feature['整体上期还款金额sum']-feature['整体上期账单金额sum']
feature['整体信用卡额度sum与整体本期账单余额sum差值']=feature['整体信用卡额度sum']-feature['整体本期账单余额sum']
feature['整体可用余额sum与整体预借现金额度sum差值']=feature['整体可用余额sum']-feature['整体预借现金额度sum']
feature['整体本期账单最低还款额sum与整体循环利息sum之和']=feature['整体本期账单最低还款额sum']+feature['整体循环利息sum']


整体账单max=gb.max()
整体账单max.columns = ['用户标识', '整体上期账单金额max', '整体上期还款金额max','整体信用卡额度max','整体本期账单余额max',
                     '整体本期账单最低还款额max','整体消费笔数max','整体本期账单金额max','整体调整金额max','整体循环利息max',
                     '整体可用余额max','整体预借现金额度max','整体还款状态max']
feature=pd.merge(feature, 整体账单max,how='left', on = "用户标识")
feature['整体上期还款金额max与整体上期账单金额max差值']=feature['整体上期还款金额max']-feature['整体上期账单金额max']
feature['整体信用卡额度max与整体本期账单余额max差值']=feature['整体信用卡额度max']-feature['整体本期账单余额max']
feature['整体可用余额max与整体预借现金额度max差值']=feature['整体可用余额max']-feature['整体预借现金额度max']
feature['整体本期账单最低还款额max与整体循环利息max之和']=feature['整体本期账单最低还款额max']+feature['整体循环利息max']

整体账单min=gb.min()
整体账单min.columns = ['用户标识', '整体上期账单金额min', '整体上期还款金额min','整体信用卡额度min','整体本期账单余额min',
                     '整体本期账单最低还款额min','整体消费笔数min','整体本期账单金额min','整体调整金额min','整体循环利息min',
                     '整体可用余额min','整体预借现金额度min','整体还款状态min']
feature=pd.merge(feature, 整体账单min,how='left', on = "用户标识")
feature['整体上期还款金额min与整体上期账单金额min差值']=feature['整体上期还款金额min']-feature['整体上期账单金额min']
feature['整体信用卡额度min与整体本期账单余额min差值']=feature['整体信用卡额度min']-feature['整体本期账单余额min']
feature['整体可用余额min与整体预借现金额度min差值']=feature['整体可用余额min']-feature['整体预借现金额度min']
feature['整体本期账单最低还款额min与整体循环利息min之和']=feature['整体本期账单最低还款额min']+feature['整体循环利息min']

整体账单mean=gb.mean()
整体账单mean.columns = ['用户标识', '整体上期账单金额mean', '整体上期还款金额mean','整体信用卡额度mean','整体本期账单余额mean',
                     '整体本期账单最低还款额mean','整体消费笔数mean','整体本期账单金额mean','整体调整金额mean','整体循环利息mean',
                     '整体可用余额mean','整体预借现金额度mean','整体还款状态mean']
feature=pd.merge(feature, 整体账单mean,how='left', on = "用户标识")
feature['整体上期还款金额mean与整体上期账单金额mean差值']=feature['整体上期还款金额mean']-feature['整体上期账单金额mean']
feature['整体信用卡额度mean与整体本期账单余额mean差值']=feature['整体信用卡额度mean']-feature['整体本期账单余额mean']
feature['整体可用余额mean与整体预借现金额度mean差值']=feature['整体可用余额mean']-feature['整体预借现金额度mean']
feature['整体本期账单最低还款额mean与整体循环利息mean之和']=feature['整体本期账单最低还款额mean']+feature['整体循环利息mean']

整体账单median=gb.median()
整体账单median.columns = ['用户标识', '整体上期账单金额median', '整体上期还款金额median','整体信用卡额度median','整体本期账单余额median',
                  '整体本期账单最低还款额median','整体消费笔数median','整体本期账单金额median','整体调整金额median',
                  '整体循环利息median','整体可用余额median','整体预借现金额度median','整体还款状态median']
feature=pd.merge(feature, 整体账单median,how='left', on = "用户标识")
feature['整体上期还款金额median与整体上期账单金额median差值']=feature['整体上期还款金额median']-feature['整体上期账单金额median']
feature['整体信用卡额度median与整体本期账单余额median差值']=feature['整体信用卡额度median']-feature['整体本期账单余额median']
feature['整体可用余额median与整体预借现金额度median差值']=feature['整体可用余额median']-feature['整体预借现金额度median']
feature['整体本期账单最低还款额median与整体循环利息median之和']=feature['整体本期账单最低还款额median']+feature['整体循环利息median']

整体账单std=gb.std()
整体账单std.columns = ['用户标识', '整体上期账单金额std', '整体上期还款金额std','整体信用卡额度std','整体本期账单余额std',
                     '整体本期账单最低还款额std','整体消费笔数std','整体本期账单金额std','整体调整金额std','整体循环利息std',
                     '整体可用余额std','整体预借现金额度std','整体还款状态std']
feature=pd.merge(feature, 整体账单std,how='left', on = "用户标识")
feature['整体上期还款金额std与整体上期账单金额std差值']=feature['整体上期还款金额std']-feature['整体上期账单金额std']
feature['整体信用卡额度std与整体本期账单余额std差值']=feature['整体信用卡额度std']-feature['整体本期账单余额std']
feature['整体可用余额std与整体预借现金额度std差值']=feature['整体可用余额std']-feature['整体预借现金额度std']
feature['整体本期账单最低还款额std与整体循环利息std之和']=feature['整体本期账单最低还款额std']+feature['整体循环利息std']

整体账单var=gb.var()
整体账单var.columns = ['用户标识', '整体上期账单金额var', '整体上期还款金额var','整体信用卡额度var','整体本期账单余额var',
                     '整体本期账单最低还款额var','整体消费笔数var','整体本期账单金额var','整体调整金额var','整体循环利息var',
                     '整体可用余额var','整体预借现金额度var','整体还款状态var']
feature=pd.merge(feature, 整体账单var,how='left', on = "用户标识")
feature['整体上期还款金额var与整体上期账单金额var差值']=feature['整体上期还款金额var']-feature['整体上期账单金额var']
feature['整体信用卡额度var与整体本期账单余额var差值']=feature['整体信用卡额度var']-feature['整体本期账单余额var']
feature['整体可用余额var与整体预借现金额度var差值']=feature['整体可用余额var']-feature['整体预借现金额度var']
feature['整体本期账单最低还款额var与整体循环利息var之和']=feature['整体本期账单最低还款额var']+feature['整体循环利息var']


In [ ]:
data=d.loc[:,['用户标识','时间','银行标识','上期账单金额', '上期还款金额','信用卡额度','本期账单余额'
              ,'本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额'
              ,'预借现金额度']].groupby(["用户标识","时间","银行标识"],as_index=False).max()

gb=data.loc[:,['用户标识', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
               '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度']].groupby(["用户标识"],as_index=False)

去重后整体账单sum=gb.sum()
去重后整体账单sum.columns = ['用户标识', '去重后整体上期账单金额sum', '去重后整体上期还款金额sum','去重后整体信用卡额度sum'
                       ,'去重后整体本期账单余额sum','去重后整体本期账单最低还款额sum','去重后整体消费笔数sum'
                       ,'去重后整体本期账单金额sum','去重后整体调整金额sum','去重后整体循环利息sum','去重后整体可用余额sum'
                       ,'去重后整体预借现金额度sum']
feature=pd.merge(feature, 去重后整体账单sum,how='left', on = "用户标识")
feature['去重后整体上期还款金额sum与整体上期账单金额sum差值']=feature['去重后整体上期还款金额sum']-feature['去重后整体上期账单金额sum']
feature['去重后整体信用卡额度sum与整体本期账单余额sum差值']=feature['去重后整体信用卡额度sum']-feature['去重后整体本期账单余额sum']
feature['去重后整体可用余额sum与整体预借现金额度sum差值']=feature['去重后整体可用余额sum']-feature['去重后整体预借现金额度sum']
feature['去重后整体本期账单最低还款额sum与整体循环利息sum之和']=feature['去重后整体本期账单最低还款额sum']+feature['去重后整体循环利息sum']


去重后整体账单max=gb.max()
去重后整体账单max.columns = ['用户标识', '去重后整体上期账单金额max', '去重后整体上期还款金额max','去重后整体信用卡额度max'
                       ,'去重后整体本期账单余额max','去重后整体本期账单最低还款额max','去重后整体消费笔数max'
                       ,'去重后整体本期账单金额max','去重后整体调整金额max','去重后整体循环利息max'
                       ,'去重后整体可用余额max','去重后整体预借现金额度max']
feature=pd.merge(feature, 去重后整体账单max,how='left', on = "用户标识")
feature['去重后整体上期还款金额max与整体上期账单金额max差值']=feature['去重后整体上期还款金额max']-feature['去重后整体上期账单金额max']
feature['去重后整体信用卡额度max与整体本期账单余额max差值']=feature['去重后整体信用卡额度max']-feature['去重后整体本期账单余额max']
feature['去重后整体可用余额max与整体预借现金额度max差值']=feature['去重后整体可用余额max']-feature['去重后整体预借现金额度max']
feature['去重后整体本期账单最低还款额max与整体循环利息max之和']=feature['去重后整体本期账单最低还款额max']+feature['去重后整体循环利息max']

去重后整体账单min=gb.min()
去重后整体账单min.columns = ['用户标识', '去重后整体上期账单金额min', '去重后整体上期还款金额min','去重后整体信用卡额度min'
                       ,'去重后整体本期账单余额min','去重后整体本期账单最低还款额min','去重后整体消费笔数min'
                       ,'去重后整体本期账单金额min','去重后整体调整金额min','去重后整体循环利息min'
                       ,'去重后整体可用余额min','去重后整体预借现金额度min']
feature=pd.merge(feature, 去重后整体账单min,how='left', on = "用户标识")
feature['去重后整体上期还款金额min与整体上期账单金额min差值']=feature['去重后整体上期还款金额min']-feature['去重后整体上期账单金额min']
feature['去重后整体信用卡额度min与整体本期账单余额min差值']=feature['去重后整体信用卡额度min']-feature['去重后整体本期账单余额min']
feature['去重后整体可用余额min与整体预借现金额度min差值']=feature['去重后整体可用余额min']-feature['去重后整体预借现金额度min']
feature['去重后整体本期账单最低还款额min与整体循环利息min之和']=feature['去重后整体本期账单最低还款额min']+feature['去重后整体循环利息min']

去重后整体账单mean=gb.mean()
去重后整体账单mean.columns = ['用户标识', '去重后整体上期账单金额mean', '去重后整体上期还款金额mean','去重后整体信用卡额度mean'
                        ,'去重后整体本期账单余额mean','去重后整体本期账单最低还款额mean','去重后整体消费笔数mean'
                        ,'去重后整体本期账单金额mean','去重后整体调整金额mean','去重后整体循环利息mean'
                        ,'去重后整体可用余额mean','去重后整体预借现金额度mean']
feature=pd.merge(feature, 去重后整体账单mean,how='left', on = "用户标识")
feature['去重后整体上期还款金额mean与整体上期账单金额mean差值']=feature['去重后整体上期还款金额mean']-feature['去重后整体上期账单金额mean']
feature['去重后整体信用卡额度mean与整体本期账单余额mean差值']=feature['去重后整体信用卡额度mean']-feature['去重后整体本期账单余额mean']
feature['去重后整体可用余额mean与整体预借现金额度mean差值']=feature['去重后整体可用余额mean']-feature['去重后整体预借现金额度mean']
feature['去重后整体本期账单最低还款额mean与整体循环利息mean之和']=feature['去重后整体本期账单最低还款额mean']+feature['去重后整体循环利息mean']


去重后整体账单median=gb.median()
去重后整体账单median.columns = ['用户标识', '去重后整体上期账单金额median', '去重后整体上期还款金额median'
                          ,'去重后整体信用卡额度median','去重后整体本期账单余额median','去重后整体本期账单最低还款额median'
                          ,'去重后整体消费笔数median','去重后整体本期账单金额median','去重后整体调整金额median'
                          ,'去重后整体循环利息median','去重后整体可用余额median','去重后整体预借现金额度median']
feature=pd.merge(feature, 去重后整体账单median,how='left', on = "用户标识")
feature['去重后整体上期还款金额median与整体上期账单金额median差值']=feature['去重后整体上期还款金额median']-feature['去重后整体上期账单金额median']
feature['去重后整体信用卡额度median与整体本期账单余额median差值']=feature['去重后整体信用卡额度median']-feature['去重后整体本期账单余额median']
feature['去重后整体可用余额median与整体预借现金额度median差值']=feature['去重后整体可用余额median']-feature['去重后整体预借现金额度median']
feature['去重后整体本期账单最低还款额median与整体循环利息median之和']=feature['去重后整体本期账单最低还款额median']+feature['去重后整体循环利息median']


去重后整体账单std=gb.std()
去重后整体账单std.columns = ['用户标识', '去重后整体上期账单金额std', '去重后整体上期还款金额std','去重后整体信用卡额度std'
                       ,'去重后整体本期账单余额std','去重后整体本期账单最低还款额std','去重后整体消费笔数std'
                       ,'去重后整体本期账单金额std','去重后整体调整金额std','去重后整体循环利息std'
                       ,'去重后整体可用余额std','去重后整体预借现金额度std']
feature=pd.merge(feature, 去重后整体账单std,how='left', on = "用户标识")
feature['去重后整体上期还款金额std与整体上期账单金额std差值']=feature['去重后整体上期还款金额std']-feature['去重后整体上期账单金额std']
feature['去重后整体信用卡额度std与整体本期账单余额std差值']=feature['去重后整体信用卡额度std']-feature['去重后整体本期账单余额std']
feature['去重后整体可用余额std与整体预借现金额度std差值']=feature['去重后整体可用余额std']-feature['去重后整体预借现金额度std']
feature['去重后整体本期账单最低还款额std与整体循环利息std之和']=feature['去重后整体本期账单最低还款额std']+feature['去重后整体循环利息std']

去重后整体账单var=gb.var()
去重后整体账单var.columns = ['用户标识', '去重后整体上期账单金额var', '去重后整体上期还款金额var','去重后整体信用卡额度var'
                       ,'去重后整体本期账单余额var','去重后整体本期账单最低还款额var','去重后整体消费笔数var'
                       ,'去重后整体本期账单金额var','去重后整体调整金额var','去重后整体循环利息var','去重后整体可用余额var','去重后整体预借现金额度var']
feature=pd.merge(feature, 去重后整体账单var,how='left', on = "用户标识")
feature['去重后整体上期还款金额var与整体上期账单金额var差值']=feature['去重后整体上期还款金额var']-feature['去重后整体上期账单金额var']
feature['去重后整体信用卡额度var与整体本期账单余额var差值']=feature['去重后整体信用卡额度var']-feature['去重后整体本期账单余额var']
feature['去重后整体可用余额var与整体预借现金额度var差值']=feature['去重后整体可用余额var']-feature['去重后整体预借现金额度var']
feature['去重后整体本期账单最低还款额var与整体循环利息var之和']=feature['去重后整体本期账单最低还款额var']+feature['去重后整体循环利息var']


In [ ]:
#爆卡指的是本期账单余额大于信用卡额度
d=bill
gb=d[(d['信用卡额度']<d['本期账单余额'])].groupby(["用户标识"],as_index=False)
x1=gb['时间'].apply(lambda x:np.unique(x).size)
x=gb['时间'].agg({'爆卡次数' : 'count'})
x['爆卡次数(去重)']=x1
feature=pd.merge(feature, x,how='left', on = "用户标识")

#用户持卡数
gb=d.groupby(["用户标识"],as_index=False)
x=gb['银行标识'].apply(lambda x:np.unique(x).size)
x1=gb['银行标识'].agg({'用户银行卡账单计数' : 'count'})
x1['用户持卡数']=x
feature=pd.merge(feature,x1,how='left', on = "用户标识")


In [ ]:
#去重银行卡标识后 信用卡额度与可用余额 
data=d.loc[:,['用户标识','银行标识','信用卡额度','可用余额','本期账单余额']].groupby(["用户标识","银行标识"],as_index=False).max()
gb=data.loc[:,['用户标识','信用卡额度','可用余额','本期账单余额']].groupby(["用户标识"],as_index=False)
去重后信用卡额度sum=gb.sum()
去重后信用卡额度max=gb.max()
去重后信用卡额度min=gb.min()
去重后信用卡额度mean=gb.mean()
去重后信用卡额度std=gb.std()
去重后信用卡额度var=gb.var()
去重后信用卡额度sum.columns = ['用户标识','去重后整体信用卡额度sum','去重后整体可用余额sum','去重后本期账单余额sum']
去重后信用卡额度max.columns = ['用户标识','去重后整体信用卡额度max','去重后整体可用余额max','去重后本期账单余额max']
去重后信用卡额度min.columns = ['用户标识','去重后整体信用卡额度min','去重后整体可用余额min','去重后本期账单余额min']
去重后信用卡额度mean.columns = ['用户标识','去重后整体信用卡额度mean','去重后整体可用余额mean','去重后本期账单余额mean']
去重后信用卡额度std.columns = ['用户标识','去重后整体信用卡额度std','去重后整体可用余额std','去重后本期账单余额std']
去重后信用卡额度var.columns = ['用户标识','去重后整体信用卡额度var','去重后整体可用余额var','去重后本期账单余额var']

feature1=pd.merge(去重后信用卡额度sum, 去重后信用卡额度max,how='outer', on = "用户标识")
feature1=pd.merge(feature1, 去重后信用卡额度min,how='left', on = "用户标识")
feature1=pd.merge(feature1, 去重后信用卡额度mean,how='left', on = "用户标识")
feature1=pd.merge(feature1, 去重后信用卡额度std,how='left', on = "用户标识")
feature1=pd.merge(feature1, 去重后信用卡额度var,how='left', on = "用户标识")

bill_feature=pd.merge(feature, feature1,how='left', on = "用户标识")

##  浏览数据表

In [ ]:
#读取数据
browse=pd.read_csv('../data/train/browse_history_train.txt',encoding='gb2312',header=None,names=['用户标识','浏览时间','浏览行为数据','浏览子行为编号'])
browse_test=pd.read_csv('../data/test/browse_history_test.txt',encoding='gb2312',header=None,names=['用户标识','浏览时间','浏览行为数据','浏览子行为编号'])
browse=pd.concat([browse,browse_test],axis=0)

#将时间戳转换为标准日期  提取month week day
browse['浏览日期'] = browse['浏览时间'].apply(lambda x :time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x)))
browse['浏览日期'] = pd.to_datetime(browse['浏览日期'])
browse['month'] = browse['浏览日期'].apply(lambda x :x.strftime('%m'))
browse['week']= browse['浏览日期'].dt.dayofweek ##输出这一天是周中的第几天，Monday=0, Sunday=6
browse['day'] = browse['浏览日期'].apply(lambda x :x.strftime('%d'))

In [ ]:
#-------------------整体特征---------------------

browse['month']=browse['month'].map(lambda x:int(x))
browse['day']=browse['day'].map(lambda x:int(x))

gb=browse.groupby(["用户标识"],as_index=False)
x1=gb['浏览子行为编号'].agg({'浏览次数' : 'count'})
x2=gb['浏览行为数据'].agg({'浏览行为数据sum' : 'sum','浏览行为数据max' : 'max','浏览行为数据mean' : 'mean',
                     '浏览行为数据min' : 'min','浏览行为数据std' : 'std','浏览行为数据var' : 'var'})
# 提取各特征出现的种数
xx1=gb['浏览子行为编号'].apply(lambda x:np.unique(x).size)
xx2=gb['浏览行为数据'].apply(lambda x:np.unique(x).size)
xx3=gb['month'].apply(lambda x:np.unique(x).size)
xx4=gb['week'].apply(lambda x:np.unique(x).size)
xx5=gb['day'].apply(lambda x:np.unique(x).size)

x2['浏览子行为编号种类']=xx1
x2['浏览行为数据种类']=xx2
x2['month种类']=xx3
x2['week种类']=xx4
x2['day种类']=xx5

#各子行为编号下的记录数
f=browse
gf=f['浏览行为数据'].groupby([f['用户标识'],f['浏览子行为编号']]).count()
gf=pd.DataFrame(gf.unstack())
name=[]
for i in range(1,12,1):
    name.append('num_子行为编号_'+str(i))
gf.columns=name

feature=pd.merge(x1,x2,how='outer',on='用户标识')
feature=pd.merge(feature,gf,how='outer',on='用户标识')

In [ ]:
#--------------时间维度下特征提取------------------

用户浏览行为=browse
#month
浏览行为数据_max =用户浏览行为['浏览行为数据'].groupby([用户浏览行为['用户标识'],用户浏览行为['month']]).max()
浏览行为数据_max = pd.DataFrame(浏览行为数据_max.unstack())


浏览行为数据_min =用户浏览行为['浏览行为数据'].groupby([用户浏览行为['用户标识'],用户浏览行为['month']]).min()
浏览行为数据_min = pd.DataFrame(浏览行为数据_min.unstack())

浏览行为数据_count =用户浏览行为['浏览行为数据'].groupby([用户浏览行为['用户标识'],用户浏览行为['month']]).count()
浏览行为数据_count = pd.DataFrame(浏览行为数据_count.unstack())

浏览行为_month_整体=pd.merge(浏览行为数据_max,浏览行为数据_min,how="left",on="用户标识")
浏览行为_month_整体=pd.merge(浏览行为_month_整体,浏览行为数据_count,how="outer",on="用户标识")


浏览行为_month_整体

In [ ]:
browse_feature=pd.merge(feature,浏览行为_month_整体,on='用户标识',how='outer')
browse_feature

##   用户基本属性表

In [ ]:
'''
这里考虑后期要进行样本记录的倒推，于是对于用户基本属性表不做特征处理，方便之后观察能否直接利用smote生成的记录充当用户基本属性记录
'''
户基本信息表=pd.read_csv("../data/train/user_info_train.txt",header=None,names=['用户标识','用户性别','用户职业',
                    '用户教育程度','用户婚姻状态','用户户口类型'])
用户基本信息表_test=pd.read_csv("../Data/test/user_info_test.txt",header=None,names=['用户标识','用户性别','用户职业',
                    '用户教育程度','用户婚姻状态','用户户口类型'])
user=pd.concat([用户基本信息表,用户基本信息表_test],axis=0)

#dataset=用户基本信息表

#label=dataset[['用户标识']]
#dataset=dataset.drop(['用户标识'],axis=1)

#dataset=pd.get_dummies(dataset,columns=['用户性别','用户职业',
                    #'用户教育程度','用户婚姻状态','用户户口类型'])

#多项式处理
#from sklearn.preprocessing import PolynomialFeatures
#poly = PolynomialFeatures(degree=2,interaction_only=True)
#dataset=poly.fit_transform(dataset)

#df = pd.DataFrame(dataset.reshape(55596,301))
#df.interpolate(axis=1).values.reshape(dataset.shape)


#label = label.reset_index(drop= True)
#df = df.reset_index(drop= True)

#user_feature=pd.concat([label,df],axis=1)

#user_feature

##   是否逾期表

In [ ]:
overdue=pd.read_csv("../Data/train/overdue_train.txt",header=None,
                    names=['用户标识','样本标签'])
overdue_test=pd.read_csv("../Data/test/overdue_test.csv")
overdue_test=overdue_test.rename(columns={'userid':'用户标识','probability':'样本标签'})

overdue=pd.concat([overdue,overdue_test],axis=0)
overdue = overdue.reset_index(drop= True)


In [ ]:
#---------所有特征表进行合并----------
train=pd.merge(bank_feature,browse_feature,on='用户标识',how='outer')
train=pd.merge(train,bill_feature,on='用户标识',how='outer')
train=pd.merge(train,user,on='用户标识',how='outer')
train=pd.merge(train,overdue,on='用户标识',how='outer')
train

In [ ]:
train.to_csv('prefore_trian.csv',encoding='gb2312')
prefore_trian = train
train['样本标签'].value_counts()

#  SMOTE样本生成

In [ ]:
'''
生成正样本
'''
train.fillna(0,inplace=True) #用0填充缺失值

smo = SMOTE(ratio={1: 16425 },random_state=42)  #生成8000条正样本
x1,y1 = train.drop('样本标签',axis=1),train[['样本标签']]
y1=y1.astype(int)
x1_smo, y1_smo = smo.fit_sample(x1, y1.values.ravel())

#提取生成新样本
smo_data1=pd.DataFrame(x1_smo)
smo_data1=smo_data1[69495:77495]
smo_data1

In [ ]:
'''
生成负样本
这里采用将原来正样本修改为'负样本'，负样本修改为'正样本'，从而生成一系列'正样本'的数据。
'''
train2=train
train2[['样本标签']]=train2[['样本标签']].replace(0,2)
train2[['样本标签']]=train2[['样本标签']].replace(1,0)
train2[['样本标签']]=train2[['样本标签']].replace(2,1)

train2.fillna(0,inplace=True) #用0填充缺失值
#生成正样本
smo = SMOTE(ratio={1: 73070},random_state=42)  #生成负样本 12000
x2,y2 = train2.drop('样本标签',axis=1),train2[['样本标签']]
y2=y2.astype(int)
x2_smo, y2_smo = smo.fit_sample(x2, y2)

#提取生成新样本
smo_data2=pd.DataFrame(x2_smo)
smo_data2=smo_data2[69495:81495]
smo_data2

In [ ]:
#正负样本合并
prefore_train=pd.read_csv('prefore_trian.csv',encoding='gb2312')
smo_data1['样本标签']=1
smo_data2['样本标签']=0
namelist=prefore_trian.columns
smo_data1.columns=namelist  #修改列名
smo_data2.columns=namelist

smo_data=pd.concat([smo_data1,smo_data2],axis=0)  #将生成的正负样本合并
smo_data=smo_data.sample(frac=1)    #打乱样本

#给生成的新样本编号  id
for i in range(1,len(smo_data)+1,1):
    smo_data.iloc[i-1,0]=i+ 69495
smo_data[['用户标识']]=smo_data['用户标识'].map(lambda x:int(x))
smo_data_Lable=smo_data[['用户标识','样本标签']]   #保存新生成样本的Lable
#smo_data_Lable.to_csv('smo_sample/smo_overdue.csv',encoding='gb2312',index=None)

## 关键特征的调整
- SMOTE 新样本关键特征存在的问题与调整方法：
-  金额的调整：
   1. 原样本中收入、支出、工资总计的金额均大于4.5，所以新样本中对应金额小于4.5的替换为0
-  缺失值的处理：
   1. 运用SMOTE生成样本过程中，利用0进行缺失值的填充，所以生成的新样本中将0替换为NaN。
-  逻辑关系的检验：
   1. 新样本中存在收入支出工资总计不为0 ，但是相应笔数却为NaN的情况。
   2. 解决方法就是找出这些记录，将相应的笔数调整为1
   

In [ ]:
#---------------------------------------------提取关键特征----------------------------------------------------
'''
用户标识 
bank关键信息：用户收入笔数、用户收入总计、用户支出笔数、用户支出总计、用户工资笔数、用户工资总计
browse关键信息：浏览次数、浏览子行为编号种类、浏览行为数据种类、month种类、weeek种类、day种类
'''
key=['用户标识','用户收入笔数','用户收入总计','用户支出笔数','用户支出总计','用户工资笔数','用户工资总计','浏览次数','浏览子行为编号种类','浏览行为数据种类','month种类','week种类','day种类']
key_features=smo_data[key]
key_features[['用户标识','用户收入笔数','用户支出笔数','用户工资笔数','浏览次数','浏览子行为编号种类','浏览行为数据种类','month种类','week种类','day种类']]=key_features[['用户标识','用户收入笔数','用户支出笔数','用户工资笔数','浏览次数','浏览子行为编号种类','浏览行为数据种类','month种类','week种类','day种类']].astype(int)
#key_features[['用户标识','用户收入笔数','用户支出笔数','用户工资笔数','浏览子行为编号种类','浏览行为数据种类','month种类','week种类','day种类']]=key_features[['用户标识','用户收入笔数','用户支出笔数','用户工资笔数','浏览子行为编号种类','浏览行为数据种类','month种类','week种类','day种类']].astype(int)
key_features.loc[key_features['用户收入总计']<4.5,'用户收入总计']=0
key_features.loc[key_features['用户支出总计']<4.5,'用户支出总计']=0
key_features.loc[key_features['用户工资总计']<4.5,'用户工资总计']=0

key_features=key_features.replace(0,np.nan)

In [ ]:
bank_key_features=key_features[['用户标识','用户收入笔数','用户收入总计','用户支出笔数','用户支出总计','用户工资笔数','用户工资总计']]
browse_key_features=key_features[['用户标识','浏览次数','浏览子行为编号种类','浏览行为数据种类','month种类','week种类','day种类']]

bank_key_features.drop_duplicates(['用户收入笔数','用户收入总计','用户支出笔数','用户支出总计','用户工资笔数','用户工资总计'],keep=False)

#--------bank--------删除整行（除银行标识）为nan 的行
bank_key_features=bank_key_features.dropna(subset=['用户收入笔数','用户收入总计','用户支出笔数','用户支出总计','用户工资笔数','用户工资总计'],how='all')

#调整工资次数为0 金额不为0 的情况（支出、收入同）
bank_key_features.loc[((bank_key_features['用户工资笔数'].isnull())&(bank_key_features['用户工资总计']>0)),'用户工资笔数']=1
bank_key_features.loc[((bank_key_features['用户收入笔数'].isnull())&(bank_key_features['用户收入总计']>0)),'用户收入笔数']=1
bank_key_features.loc[((bank_key_features['用户支出笔数'].isnull())&(bank_key_features['用户支出总计']>0)),'用户支出笔数']=1


#------browse-------删除只要包含nan的行
browse_key_features=browse_key_features.dropna(subset=['浏览次数','浏览子行为编号种类','浏览行为数据种类','month种类','week种类','day种类'],how='any')

In [ ]:
#bank_key_features.to_csv('smo_sample/bank_key_features.csv',encoding='gb2312',index=None)
#browse_key_features.to_csv('smo_sample/browse_key_features.csv',encoding='gb2312',index=None)

# 生成新的浏览记录
## 具体步骤
- 定义关键信息：
      ID:用户标识
      N:浏览次数
      n1:子行为编号种类
      n2:浏览行为数据种类
      n3:月份种数
      n4:星期种数
      n5:日期种数
    
- 准备各特征的取值总集：
      子行为编号种类A，
      浏览行为数据种类B，
      月份Cm，
      星期Cw，
      日期Cd
   
- 生成单个id的浏览特征集合（不放回）
   1. 从A中依照原样本分布概率抽样抽取n1个，构成A1
   2. 从B中依照原样本分布概率抽样抽取n2个，构成B1
   3. 从Cm中依照原样本分布概率抽样抽取n3个，构成C1
   4. 从Cw、Cd中分别随机抽样抽取n4、n5个，构成C2、C3
   
- 生成浏览记录（放回）
   1. 从A1中依概率抽取N个构成：子行为编号
   2. 从B1中依概率取N个构成：浏览行为数据
   3. 从C1中依概率抽取N个构成：month
   4. 从C2、C3中分别依概率抽取N个构成：week、day
 
- 生成所有id的记录合并在一起

In [ ]:
#-----------原数据特征准备------
browse_key_features=pd.read_csv('smo_sample/browse_key_features.csv',encoding='gb2312')
browse_key_features[['用户标识','浏览次数','浏览子行为编号种类','浏览行为数据种类','month种类','week种类','day种类']]=browse_key_features[['用户标识','浏览次数','浏览子行为编号种类','浏览行为数据种类','month种类','week种类','day种类']].astype(int)

browse_train=pd.read_csv('../data/train/browse_history_train.txt',encoding='gb2312',header=None,names=['用户标识','浏览时间','浏览行为数据','浏览子行为编号'])
browse_test=pd.read_csv('../data/test/browse_history_test.txt',encoding='gb2312',header=None,names=['用户标识','浏览时间','浏览行为数据','浏览子行为编号'])
browse=pd.concat([browse_train,browse_test],axis=0)

browse['浏览日期'] = browse['浏览时间'].apply(lambda x :time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x)))
browse['浏览日期'] = pd.to_datetime(browse['浏览日期'])
browse['month'] = browse['浏览日期'].apply(lambda x :x.strftime('%m'))
browse['week']= browse['浏览日期'].dt.dayofweek ##输出这一天是周中的第几天，Monday=0, Sunday=6
browse['day'] = browse['浏览日期'].apply(lambda x :x.strftime('%d'))

In [ ]:
A=list(browse['浏览子行为编号'].unique())
B=list(browse['浏览行为数据'].unique())
Cm=list(list(range(1,13,1)))
Cw=list(range(0,7,1))
Cd=list(range(1,32,1))

#浏览子行为编号的比重
Pa=[1644330,913855,1132344,1468039,1742281,1975832,1376926,2688287,3387346, 4824424,4198328]
Pa=np.array(Pa)
Pa=Pa/sum(Pa)

#浏览行为数据的比重
Pb=Counter(browse['浏览行为数据'])
a=sorted(Pb.items(),key=lambda x:x[0],reverse=False)
Pb=[]
for i in range(216):
    Pb.append(a[i][1])
Pb=np.array(Pb)
Pb=Pb/sum(Pb)

#month 的概率比重
Pm=Counter(browse['month'])
a=sorted(Pm.items(),key=lambda x:x[0],reverse=False)
Pm=[]
for i in range(12):
    Pm.append(a[i][1])
Pm=np.array(Pm)
Pm=Pm/sum(Pm)

def single_id(ID,N,n1,n2,n3,n4,n5):
    
    #生成单个id的特征总集
    A1=np.random.choice(A,n1,replace=False,p=Pa)
    B1=np.random.choice(B,n2,replace=False,p=Pb)
    C1=np.random.choice(Cm,n3,replace=False,p=Pm)
    C2=np.random.choice(Cw,n4,replace=False)
    C3=np.random.choice(Cd,n5,replace=False)
    
    #二重概率抽样的概率
    Pa1 = Pa[A1-1]
    Pa1=Pa1 /sum(Pa1)
    Pb1 = Pb[B1-1]
    Pb1=Pb1 /sum(Pb1)
    Pm1 = Pm[C1-1]
    Pm1=Pm1 /sum(Pm1)
    
    #生成各特征取值list
    子行为编号=np.random.choice(A1,N,replace=True,p=Pa1)
    浏览行为数据=np.random.choice(B1,N,replace=True,p=Pb1)
    month=np.random.choice(C1,N,replace=True,p=Pm1)
    week=np.random.choice(C2,N,replace=True)
    day=np.random.choice(C3,N,replace=True)

    c={'用户标识':ID,'浏览子行为编号':子行为编号,'浏览行为数据':浏览行为数据,'month':month,'week':week,'day':day}
    c=pd.DataFrame(c)
    return c

def  get_browse(d):
    #提取每个id的关键信息
    ID = d['用户标识']
    N = d['浏览次数']
    n1 = d['浏览子行为编号种类']
    n2 = d['浏览行为数据种类']
    n3 = d['month种类']
    n4 = d['week种类']
    n5 = d['day种类']
    c=single_id(ID,N,n1,n2,n3,n4,n5)
    print(c)
    #c=pd.concat([smo_browse,c],axis=0)
    #print(c)
    return c
    
smo_browse=pd.DataFrame(columns=('用户标识','浏览子行为编号','浏览行为数据','month','week','day'))

if __name__ == '__main__':   
    smo_browse= bf.apply(get_browse , axis = 1)


In [ ]:
#合并apply出来的单个id 的浏览记录
sss=pd.DataFrame(smo_browse)
smo_browse1=pd.DataFrame(columns=('用户标识','浏览子行为编号','浏览行为数据','month','week','day'))
for i in range(len(bf)):
    browse_id=pd.DataFrame(sss.iloc[i,0])
    a=i
    #print(a)
    smo_browse1=pd.concat([smo_browse1,browse_id],axis=0)

# 生成新的银行流水记录
## 具体步骤
- 定义各关键信息
       n1：收入的次数
       V1：收入总计
       n2：工资笔数
       v2：工资总计
       n3：支出笔数
       V3：支出总计
       
-  生成工资表： n2工资笔数、v2工资总计
   1. 生成时间戳
     判断n2<=18？   
          是，则生成n2个月的工资时间戳（每月同一时间）
          否，则先生成18个月的工资时间戳，多于18的工资时间戳随机生成
   2. 生成工资记录
     生成n2个随机数（这里取[10,20]），调整其和为v2
   3. 交易类型：0；工资标记：1
   
- 生成收入表：n1收入的次数，V1收入总计
   1. 在时间范围内随机生成时间戳（n1-n2）个
   2. 交易金额：生成（n1-n2）个随机数（这里取[10,20]），调整其和为（v1-v2）
   3. 交易类型：0；工资标记：0
   
- 生成支出表：n3支出笔数，V3支出总计
   1. 在时间范围内随机生成时间戳n3个
   2. 交易金额：生成n3个随机数（这里取[10,20]），调整其和为v3
   3. 交易类型：1；工资标记：0

In [ ]:
k=bank_key_features
#-----------工资表---------------
def salary(ID,n2,v2):
    
    工资表=pd.DataFrame(columns=('用户标识','流水时间','交易类型','交易金额','工资收入标记'))
    #生成时间戳
    if np.isnan(n2):
        工资表=工资表
    else:
        n2=int(n2)
        i=random.randint(0,31)
        end_time=5928939087-i*60*60*24  #工资时间都是出现在当天的7:31分，为保持一致随机生成最后一月随机一天的7:31作为工资每月发放时间点
        
        if n2<=18:   #原有样本时间跨度差不多在18个月左右，于是进行判断
            salary_time=[]
            for i in range(n2):
                salary_time.append(end_time-30*24*60*60*i)
        else:
            salary_time1=[]
            for i in range(18):
                salary_time1.append(end_time-30*24*60*60*i)
            n=n2-18      #对于大于18个月的ID，多于的时间点就随机生成
            salary_time2=[random.randint(5879518000,5928983000) for _ in range(n)]#random.randomint(5879518000,5928983000,n)
            salary_time=salary_time1+salary_time2
        
        #生成工资金额序列
        salary=np.random.randint(10,20,n2)
        ratio=v2/sum(salary)
        salary=list(salary*ratio)

        工资表={'用户标识':ID,'流水时间':salary_time,'交易类型':0,'交易金额':salary,'工资收入标记':1}
        工资表=pd.DataFrame(工资表)
    return 工资表
#-------------收入表- -----------
def income(ID,n1,v1,n2,v2):
    收入表=pd.DataFrame(columns=('用户标识','流水时间','交易类型','交易金额','工资收入标记'))
    n=n1-n2
    if np.isnan(n):
        收入表=收入表
    else:
        if n==0:
            收入表=收入表
        else:
            n=int(n)
            income_time=[random.randint(5879518000,5928983000) for _ in range(n)] #random.randomint(5879518000,5928983000,n)        
           #生成工资金额序列
            income=np.random.randint(10,20,n)
            ratio=(v1-v2)/sum(income)
            income=list(income*ratio)

            收入表={'用户标识':ID,'流水时间':income_time,'交易类型':0,'交易金额':income,'工资收入标记':0}
            收入表=pd.DataFrame(收入表)
    return 收入表
#-------------支出表---------------
def outcome(ID,n3,v3):
    支出表=pd.DataFrame(columns=('用户标识','流水时间','交易类型','交易金额','工资收入标记'))

    if np.isnan(n3):
        支出表=支出表
    else:
        n3=int(n3)
        outcome_time=[random.randint(5879518000,5928983000) for _ in range(n3)]  #random.randomint(5879518000,5928983000,n)      
    #生成工资金额序列
        outcome=np.random.randint(10,20,n3)
        ratio=v3/sum(outcome)
        outcome=list(outcome*ratio)

        支出表={'用户标识':ID,'流水时间':outcome_time,'交易类型':1,'交易金额':outcome,'工资收入标记':0}
        支出表=pd.DataFrame(支出表)
        
    return 支出表

def get_feature(k):
 #n1收入的次数，V1收入总计；n2工资笔数，v2工资总计；n3支出笔数，V3支出总计
    ID=k['用户标识']
    n1=k['用户收入笔数']
    v1=k['用户收入总计']
    n2=k['用户工资笔数']
    v2=k['用户工资总计']
    n3=k['用户支出笔数']
    v3=k['用户支出总计']
    
    工资表 = salary(ID,n2,v2)
    收入表 = income(ID,n1,v1,n2,v2)
    支出表 = outcome(ID,n3,v3)
    
    bank_id = pd.concat([工资表,收入表],axis=0)
    bank_id = pd.concat([bank_id,支出表],axis=0)
    bank_id= bank_id.sort_values(by='流水时间')
    
    return bank_id

if __name__ == '__main__':   
    smo_bank= k.apply(get_feature , axis = 1)

In [ ]:
#将apply出来的表进行合并
smo_bank= pd.DataFrame(smo_bank)
smo_bank1=pd.DataFrame(columns=('用户标识','流水时间','交易类型','交易金额','工资收入标记'))

for i in range(len(k)):
    bank_id=pd.DataFrame(smo_bank.iloc[i,0])
    #print(i)
    smo_bank1=pd.concat([smo_bank1,bank_id],axis=0)

# 生成新的用户基本信息表
## 具体步骤
用户基本信息表中包含六个字段：用户标识、用户性别、用户职业、用户教育程度、用户婚姻状态、用户户口类型
- 生成新正样本的用户基本信息表
  1. 从原有正样本用户基本信息表中，分别从除id的5个基本特征构成的5个集合中随机抽取8000条，构成最终新正样本的用户基本信息。
  2. 提取smo_overdue中新正样本的id，从而生成新正样本中用户基本信息的id
- 生成新负样本的用户基本信息表：
  1. 从原有负样本用户基本信息表中，分别从除id的5个基本特征构成的5个集合中随机抽取12000条，构成最终新正样本的用户基本信息。
  2. 提取smo_overdue中新负样本的id，从而生成新负样本中用户基本信息的id
  


In [ ]:
'''
读取数据、并划分正负样本分别对应的用户基本信息表
'''
smo_overdue=pd.read_csv('smo_sample/smo_overdue.csv',encoding='gb2312')

user=pd.read_csv("../data/train/user_info_train.txt",header=None,names=['用户标识','用户性别','用户职业',
                    '用户教育程度','用户婚姻状态','用户户口类型'])
user_test=pd.read_csv("../data/test/user_info_test.txt",header=None,names=['用户标识','用户性别','用户职业',
                    '用户教育程度','用户婚姻状态','用户户口类型'])
user=pd.concat([user,user_test],axis=0)

overdue=pd.read_csv("../Data/train/overdue_train.txt",header=None,names=['用户标识','样本标签'])
overdue_test=pd.read_csv("../Data/test/overdue_test.csv")
overdue_test=overdue_test.rename(columns={'userid':'用户标识','probability':'样本标签'})
overdue=pd.concat([overdue,overdue_test],axis=0)
overdue = overdue.reset_index(drop= True)
user = pd.merge(user,overdue,on='用户标识',how='outer')

smo_overdue_1=smo_overdue[smo_overdue['样本标签']==1]
smo_overdue_0=smo_overdue[smo_overdue['样本标签']==0]

user_1=user[user['样本标签']==1]
user_0=user[user['样本标签']==0]

In [ ]:
#--------------------对于label=0  生成user基本信息-------------------
#从原负样本的各个特征列中 随机抽取12000条数据构成新id负样本的特征

data= user_0
ID=smo_overdue_0['用户标识']
用户性别 = np.random.choice(data['用户性别'], 12000)
用户职业 = np.random.choice(data['用户职业'], 12000)
用户教育程度 = np.random.choice(data['用户教育程度'], 12000)
用户婚姻状态 = np.random.choice(data['用户婚姻状态'], 12000)
用户户口类型 = np.random.choice(data['用户户口类型'], 12000)

smo_user_0={'用户标识':ID,'用户性别':用户性别,'用户职业':用户职业,'用户教育程度':用户教育程度,'用户婚姻状态':用户婚姻状态,'用户户口类型':用户户口类型}
smo_user_0=pd.DataFrame(smo_user_0)

#--------------------对于label=1  生成user基本信息-------------------
#从原正样本的各个特征列中 随机抽取8000条数据构成新id正样本的特征
data= user_1
ID=smo_overdue_1['用户标识']
用户性别 = np.random.choice(data['用户性别'], 8000)
用户职业 = np.random.choice(data['用户职业'], 8000)
用户教育程度 = np.random.choice(data['用户教育程度'], 8000)
用户婚姻状态 = np.random.choice(data['用户婚姻状态'], 8000)
用户户口类型 = np.random.choice(data['用户户口类型'], 8000)

smo_user_1={'用户标识':ID,'用户性别':用户性别,'用户职业':用户职业,'用户教育程度':用户教育程度,'用户婚姻状态':用户婚姻状态,'用户户口类型':用户户口类型}
smo_user_1=pd.DataFrame(smo_user_1)

smo_user=pd.concat([smo_user_0,smo_user_1],axis=0)
smo_user=smo_user.sort_values(by='用户标识')
#smo_user.to_csv('smo_sample/smo_user.csv',encoding='gb2312',index=None)

# 处理信用卡账单记录表
 处理信用卡账单数据  
- 问题：  
  1. 时间错乱
  2. 数字太乱  
  3. 可能出现账单的断裂
- 目标：  
  1. 归序时间  
  2. 优化数字的显示方式  
  3. 拼接账单链
- 步骤：
  1. 排序-为每个用户每张信用卡的账单  
  2. 完善账单链，并补充时间，间隔先设置为30天
  3. 更改金额数字

In [ ]:
import pandas as pd
import numpy as np
import copy

# 变量定义
N = 20000           # 定义新生成样本的数量
LimitBanks = 4      # 每个人最多有4个银行卡
indBenQi = 6        # 本期账单余额所在列
indShangQi = 3      # 上期账单金额所在列
indShangQiHuan = 4  # 上期还款金额所在列
indEdu = 5          # 信用卡额度所在列
indTime = 2         # 时间戳所在的列
pAddrec = 0.8       # 加入新纪录作为粘合两条断裂链的概率
newidStart = 69495  # 新样本的编号，从这个数字开始

## 数据预处理
- 有重复值。$\color{red}{删除}$重复值
- 不同银行简单统计。
  1. 因为数据来自不同银行，会发现一些银行的数据比较诡异，故需查看。  
    bankid = 1，3100条记录，缺失值非常多    
    bankid = 2，13.5万条记录，相对质量较高  
    bankid = 3，22.1万条记录，基本没有上期账单金额和还款金额  
    bankid = 4，26.5万条记录  
    bankid = 510014和510044，有循环利息，但数据量较少
  2. $\color{red}{删除}$部分银行的记录。  
    编号为1、12，以及所有大于50000的银行id，数量都较少。
- 需直接$\color{red}{删除}$的字段
  1. 消费笔数：经常出现消费笔数为0，但账单余额不为0的情况。  
  2. 可用余额：出现数量非常少，在某个银行id下会较多。但此数字非常奇怪。  
  3. 调整金额，循环利息，预借现金额度：基本上都是空值，仅在少量小众银行下有值。   
  4. 本期账单金额：数值非常奇怪，感觉跟其余的字段完全没有关系。  
  5. 本期账单最低还款额：基本没有什么用处。  
- 需判断后处理的记录
  1. 信用卡额度：有些用户会有较大波动，估计是数据合并时出现错误。需检查波动情况，然后删除此用户此银行的所有记录。

In [ ]:
'''
  第1步，读入数据，删除无用字段，并删除重复值
'''
datapath = '../data/'
train_credit = pd.read_csv(datapath + 'train/bill_detail_train.txt', header = None)
test_credit = pd.read_csv(datapath + 'test/bill_detail_test.txt', header = None)
train_credit = pd.concat([train_credit,test_credit],axis = 0)
train_credit.columns = ['用户id','账单时间戳','银行id','上期账单金额','上期还款金额','信用卡额度','本期账单余额',  
                     '本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额',  
                     '预借现金额度','还款状态']
print('原始数据总规模为：', train_credit.shape)
remvVars = ['消费笔数','可用余额','调整金额','循环利息','预借现金额度','本期账单金额', '本期账单最低还款额']
train_credit.drop(remvVars, axis=1, inplace=True)
train_credit.drop_duplicates(inplace=True)
print('删除无用字段和重复值之后的数据总规模为：', train_credit.shape)
train_credit.drop
train_credit.head()

In [ ]:
'''
  第2步，删除指定银行的数据
'''
remvBankid = [1,3,12,510014,510016,510017,510022,510024,510025,510026,510027,510033,510037,510044,510050,510053,510057]
train_credit = train_credit.set_index(['银行id'])
train_credit.drop(remvBankid,inplace=True)
train_credit.reset_index()

In [ ]:
'''
  第3步，删除信用卡额度变化太快的字段
  
  方法：取出每一对（用户id，银行id）对应的记录，然后判断其众数，若众数数量不超过
  min(6,总记录数量的一半)，则不保留此数据
'''
# 新建一个df，存放被保留的数据
temp_df = pd.DataFrame()
# 取出所有用户id
user_lst = list(set(train_credit['用户id'].tolist()))
i = 1
for usr in user_lst:
    i += 1
    if i % 1000 == 0:
        print('已完成处理%d个用户...'%i)
    # 取出用户usr的所有记录
    usr_df = train_credit[train_credit['用户id']==usr]
    # 取出用户usr的所有银行id
    bank_lst = list(set(usr_df['银行id'].tolist()))
    for bank in bank_lst:
        # 取出用户usr&银行bank的所有记录
        usr_bank_df = usr_df[usr_df['银行id']==bank]
        if usr_bank_df.shape[0] < 3:
            temp_df = pd.concat([temp_df,usr_bank_df], axis=0)
        else:
            # 求得“信用卡额度”的众数出现的次数
            limit_count = usr_df['信用卡额度'].value_counts()
            limit_mostFreqCount = list(limit_count)[0]
            if limit_mostFreqCount < min(6,int(usr_df['信用卡额度'].shape[0]/2)):
                # 跳过这个usr_bank_df
                pass
            else:
                temp_df = pd.concat([temp_df,usr_bank_df], axis=0)
        
print('删除异常信用卡额度之后的数据总规模为：', temp_df.shape)        

In [ ]:
'''
  最后，把处理好的数据写入文件，备用
''' 
temp_df.to_csv('Billdata_clean_20190817.csv')

## 排序-单个用户单个银行
- 方法：
  1. 以原数据的时间作为基本排序
  2. 根据“上期账单账单金额”与“本期账单金额”的对应关系，优化时间序列
  3. 若存在断链的情况，则找出所有可以连起来的链；再做进一步的判断
- 处理流程：  
  请见同文件下文件名：农业银行数据-信用卡账单处理流程图

In [ ]:
# 1. 读入新数据
bill_df = pd.read_csv('Billdata_clean_20190817.csv')
bill_df.head()

In [ ]:
# 2 定义排序函数
def sortBillList(df_usr_bank):
    # 由于dataframe类型，不便于重复删除指定行，且控制好指针，故转为二维数组
    arr = df_usr_bank.sort_values(by='账单时间戳').values
        
    # bill链可能会出现断裂，此时会找到所有连续的链，存放于resL。
    resL = []             # resL定义为一个三维数组
    while True:
        if len(arr) == 0:
            return resL
        elif len(arr) == 1:
            L = [list(arr[0])]
            resL += [L,]
            return resL
        else:
            L = []
            L = L + [list(arr[0]),]   #在L中加入arr中的第一条 然后进行判断
            arr = np.delete(arr,0,axis=0)
            i = 0
            
            while i < len(arr):
                if arr[i][indShangQi] == L[len(L)-1][indBenQi]:   
                    L = L + [list(arr[i]),]
                    arr = np.delete(arr,i,axis=0)
                    i = 0
                elif arr[i][indBenQi] == L[0][indShangQi]:   
                    L = [list(arr[i]),] + L
                    arr = np.delete(arr,i,axis=0)
                    i = 0
                else:
                    i += 1
            # 然后判断df是否为空
            resL = resL + [L,]   

In [ ]:
# 提出所有用户id； 建议分几批来运行，速度会快很多。每次生成的数据，保存成npy即可。
userSet = list(set(bill_df['用户id'].tolist()))

allSortedList = []

# 遍历每一位用户
i = 1
for usr in userSet:
    if i % 1000 == 0:
        print('已处理%d个用户...'%i)
    i += 1
    df_usr = bill_df.loc[bill_df['用户id'] == usr]
    # 遍历此用户的每一个银行id
    bankSet = set(df_usr['银行id'].tolist())
    for bank in bankSet:
        df_usr_bank = df_usr.loc[df_usr['银行id'] == bank]
        # 调用排序算法,返回生成该“用户&银行”的所有list，存入allSortedList
        allSortedList = allSortedList + [sortBillList(df_usr_bank)]
    

# 将数据保存
np.save('tempBillChain.npy',allSortedList)

## 合并BillChains
- 背景  
  对于当前的数据，对用户u银行i的所有信用卡账单，断裂非常严重，需要补成链状。  
- 合并的顺序  
  以信用卡额度为标准，小额度的尽可能放前面。
- 方法  
  1. 按较高的概率，添加一条记录，把两条断裂的链粘合起来
  2. 按较小的概率，直接把两条链放在一起，就当作是记录缺失了  
  
  具体的方案，可以看当前目录下的流程图，合并Billchains.jpg

In [ ]:
# 读入排好顺序的BillChain
allSortedList = np.load('tempBillChain.npy')

In [ ]:
# 获得每一个连续链的第一个非0信用卡额度值
def getValidCreditLimit(lst2d, ind):
    res = 0
    for templst in lst2d:
        if templst[ind] > res:
            res = templst[ind]
            break
    return res

# 开始合并
mergedChains = []
iout = 1
for usr_bank_list in allSortedList:
    if iout % 1000 == 0:
        print('已处理%d个用户...'%iout)
    iout += 1
    
    resL = usr_bank_list[0]
    # 获取当前resL中首个大于0的额度
    begCreditLimit = getValidCreditLimit(resL, indEdu)
            
    for i in range(1,len(usr_bank_list)):
        # 获取当前usr_bank_list[i]记录中，第一个不为0的额度
        thisCreditLimit = getValidCreditLimit(usr_bank_list[i], indEdu)
        
        if thisCreditLimit < begCreditLimit:           
            # 把第i条记录加在resL“头部”
            if random.random() < pAddrec:
                # 新增记录，链接resL[0]和usr_bank_list[i]
                newrecord = usr_bank_list[i][len(usr_bank_list[i])-1][:]
                # 新增记录的上期金额 = usr_bank_list[i][end][本期]
                newrecord[indShangQi] = usr_bank_list[i][len(usr_bank_list[i])-1][indBenQi]   
                # 新增记录的本期余额 = resL[0][上期]
                newrecord[indBenQi] = resL[0][indShangQi]                 
                # 新增记录的上期还款金额，在上期金额上，进行浮动，[-0.2，0.2]
                newrecord[indShangQiHuan] = newrecord[indShangQi] + (random.random()-0.5)/2
                resL = [newrecord] + resL
            resL = usr_bank_list[i] + resL
        else:
            # 把第i条记录加在resL“尾部”
            if random.random() < pAddrec:
                # 新增记录，链接resL[end]和usr_bank_list[i]
                newrecord = resL[len(resL)-1][:]
                # 新增记录的上期金额 = resL[end][本期]
                newrecord[indShangQi] = resL[len(resL)-1][indBenQi] 
                # 新增记录的本期余额 = usr_bank_list[i][0][上期]
                newrecord[indBenQi] = usr_bank_list[i][0][indShangQi]  
                # 新增记录的上期还款金额，在上期金额上，进行浮动，[-0.2，0.2]
                newrecord[indShangQiHuan] = newrecord[indShangQi] + (random.random()-0.5)/2
                resL = resL + [newrecord]
            resL = resL + usr_bank_list[i]
            begCreditLimit = thisCreditLimit
    
    mergedChains = mergedChains + [resL,]

# 把得到的结果保存下来，原始数据的记录
np.save('initialUserBillChainP'+str(pAddrec)+'.npy',mergedChains)

## 生成新样本
- 目的与原则
  1. 补充新的样本数据
  2. 需要区分正负样本，12000个负样本，8000个正样本
- 方法
  1. 为每一个用户，安排1~4个银行
  2. 对于新的正样本用户，仅选择正样本用户的信用卡账单情况；负样本用户同样仅对应负样本
  3. 在原有记录上，仅删除样本数据  
  
  具体的删除样本方案，可以看当前目录下的流程图，生成新样本.jpg

In [ ]:
# 先读入数据
initialBillChains = np.load('initialUserBillChainP'+str(pAddrec)+'.npy')
# 根据标签数据 'overdue_train.txt'，将initialBillChains分成两部分
labels = pd.read_csv('../data/train/overdue_train.txt',header=None)
labels.columns = ['用户id','label']

iniBCpos, iniBCneg = [],[]
ii = 1
for lst in initialBillChains:
    if ii % 1000 == 0:
        print('已完成%d条，共%d条...'%(ii,len(initialBillChains)))
    ii += 1
    usrid = lst[0][1]
    try:
        if labels[labels['用户id']==usrid]['label'].tolist()[0] == 1:
            iniBCpos = iniBCpos + [lst]
        else:
            iniBCneg = iniBCneg + [lst]
    except:
        pass

In [ ]:
def GenerateSamples(iniBC,idStart,Nsample):
    newUserBC = []
    for i in range(Nsample):
        if (i+1)%500 == 0:
            print('已新建%d个用户...'%i)
        r = np.random.randint(LimitBanks) + 1
        newid = idStart + i
        banks = []
        for _ in range(r):
            # 随机取出一条记录，应是一个二维数组
            tempind =  np.random.randint(len(iniBC))
            thisChain = copy.deepcopy( iniBC[tempind] )
            # 取出这条记录的bankid
            bankid = thisChain[0][0]
            if bankid in banks:
                continue
            # 开始改造这条记录
            banks.append(bankid)

            ####### 改造1：删除记录
            if np.random.random() < 0.5:
                # 只删除头或尾
                x = min(7,int((len(thisChain)-1)/3) )
                xhead = int(x/2)
                xtail = x-xhead
                thisChain = np.delete(thisChain,range(xhead),axis=0)
                thisChain = np.delete(thisChain,range(len(thisChain)-xtail,len(thisChain)),axis=0)
            else:
                # 混合删除
                a = min(7,int((len(thisChain)-1)/3) )
                y = min(3,a)
                if a>3:
                    x = a-3
                    xhead = int(x/2)
                    xtail = x-xhead
                    thisChain = np.delete(thisChain,range(xhead),axis=0)
                    thisChain = np.delete(thisChain,range(len(thisChain)-xtail,len(thisChain)),axis=0)
                # 删除y个中间值
                remvInds = np.random.randint(0,len(thisChain),y)
                thisChain = np.delete(thisChain,remvInds,axis=0)

            ####### 改造2：替换userid
            thisChain[:,1] = newid

            ####### 改造3：上期金额、本期余额、信用卡额度，统一添加一个随机数
            ################### 但原来为0的，不能加
            temprand = (np.random.random() - 0.5)/2
            thisChain[:,indBenQi] = thisChain[:,indBenQi] + temprand
            thisChain[:,indShangQi] = thisChain[:,indShangQi] + temprand
            thisChain[:,indEdu] = thisChain[:,indEdu] + temprand

            ####### 改造4：上期还款金额，各自改一个小小的随机数
            thisChain[:,indShangQiHuan] = thisChain[:,indShangQiHuan] + temprand/10
            
            thisChain[(thisChain < 0.5) & (thisChain>-0.5)] = 0
            
            newUserBC = newUserBC + [thisChain,]
    return newUserBC

In [ ]:
newBCneg = GenerateSamples(iniBCneg,newidStart,12000)
newBCpos = GenerateSamples(iniBCpos,newidStart+12000,8000)
np.save('newUserBCneg.npy',newBCneg)
np.save('newUserBCpos.npy',newBCpos)

## 样本进一步加工
- 方法
  1. 以较大概率，0.6，将一些负值置为0  
  2. 若时间戳为0，则以0.5的概率将此条记录删除  
  3. 若时间戳不为0，则在原来时间上做一些修改，找到第一个不为0的时间，往后看每一条记录  
     3.1 若与前一条连续，则加(29.8~31.2)天的偏移。  
     3.2 如不连续，则加(29.8~31.2)天的偏移。  
     注：当前时间戳，单位为秒，因此需要 $随机数*24*3600$

In [ ]:
def Process(billChains):
    thisBC = []
    for lst in billChains:
        # 遍历此记录
        lst = np.array(lst)
        i = 0
        while i < len(lst):
            if lst[i][indTime] == 0:
                if np.random.random() < 0.5:
                    lst = np.delete(lst, i, axis=0)
                    i -= 1
                elif np.random.random() < 0.6:
                    # 将负值置为0
                    lst[lst < 0] = 0
            else:
                if np.random.random() < 0.6:
                    # 将负值置为0
                    lst[lst < 0] = 0
                try:
                    if lst[i][indBenQi] == lst[i+1][indShangQi]:
                        timelag = int((np.random.random()*(31.2-29.8) + 29.8) * 24 * 3600)
                        lst[i+1][indTime] = lst[i][indTime] + timelag
                    else:
                        timelag = int((np.random.random()*(31.2-29.8) + 29.8) * 24 * 3600 * np.random.randint(2,4))
                        lst[i+1][indTime] = lst[i][indTime] + timelag
                except:
                    pass
            i += 1
        thisBC = thisBC + [lst]
    return thisBC

In [ ]:
# 处理newUserBCpos
newBCpos = np.load('newUserBCpos.npy')
newBCpos1 = Process(newBCpos)
# 处理newUserBCneg
newBCneg = np.load('newUserBCneg.npy')
newBCneg1 = Process(newBCneg)
# 处理iniBC
iniBC = np.load('initialUserBillChainP0.8.npy')
iniBC1 = Process(iniBC)
print('done')

## 将array还原成dataframe

In [ ]:
np.set_printoptions(suppress=True)
def arrayToDF(arr):
    res = []
    for lst in arr:
        res = res + list(lst)
    res_df = pd.DataFrame(res)
    res_df.columns = ['银行id','用户id','账单时间戳','上期账单金额','上期还款金额','信用卡额度','本期账单余额','还款状态']
    return res_df

In [ ]:
newBCpos_df = arrayToDF(newBCpos1)
newBCpos_df.to_csv('newBill_pos.csv', index = 0)

newBCneg_df = arrayToDF(newBCneg1)
newBCneg_df.to_csv('newBill_neg.csv', index = 0)

## 由于过摸较大要分开写，运行才会比较快，比如下面的可替换代码
iniBC_df = arrayToDF(iniBC1)
iniBC_df.to_csv('Bill.csv', index = 0)


In [ ]:
# 可替换的代码
iniBC1_1 = iniBC1[0:30000]
iniBC_df1 = arrayToDF(iniBC1_1)
iniBC1_2 = iniBC1[30000:60000]
iniBC_df2 = arrayToDF(iniBC1_2)
#诸如此类